In [7]:
import json
import requests
import csv
import os
from tqdm import tqdm

In [8]:
def get_wikipedia_title_from_wikidata(qid: str, language: str = "en") -> str:
    """
    Given a Wikidata QID (e.g., 'Q328656'), retrieve the corresponding
    Wikipedia page title in the specified language (default: English).
    Returns None if the item does not have a matching sitelink or if there's an error.
    """
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching Wikidata for {qid}.")
        return None
    
    data = response.json()
    try:
        entity_info = data["entities"][qid]
        sitelinks = entity_info["sitelinks"]
        wiki_key = f"{language}wiki"
        if wiki_key in sitelinks:
            return sitelinks[wiki_key]["title"]
        else:
            print(f"No '{language}wiki' sitelink for {qid}")
            return None
    except KeyError:
        print(f"Wikidata entry not found or malformed for {qid}")
        return None
    
def gather_qids_from_files(file_paths):
    """
    Given a list of JSON file paths, gather all QIDs (main 'wikidata_ID'
    plus any in 'answer_ids'). Returns a set of QIDs.
    """
    all_qids = set()
    for path in file_paths:
        if not os.path.exists(path):
            print(f"Warning: File {path} does not exist. Skipping.")
            continue
        
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
            for item in data:
                # Add main wikidata_ID
                if "wikidata_ID" in item:
                    all_qids.add(item["wikidata_ID"])
                
                # Add any QIDs from 'answer_ids'
                if "answer_ids" in item:
                    for _, v in item["answer_ids"].items():
                        all_qids.add(v)
    return all_qids

In [9]:
# Update these paths as needed
json_file_paths = [
    "../data/train_TLQA.json",  # or "train_TLQA.json"
    "../data/test_TLQA.json",   # or "test_TLQA.json"
    "../data/val_TLQA.json"     # or "val_TLQA.json"
]

# 1. Gather all QIDs
qid_set = gather_qids_from_files(json_file_paths)
print(f"Found {len(qid_set)} unique QIDs in total.")

# 2. For each QID, retrieve the Wikipedia title (with a progress bar)
qid_title_pairs = []
for qid in tqdm(qid_set, desc="Retrieving Wikipedia titles"):
    title = get_wikipedia_title_from_wikidata(qid, language="en")
    if title:
        qid_title_pairs.append((qid, title))

print(f"Successfully retrieved {len(qid_title_pairs)} titles.")

# 3. Write out to CSV
output_csv_path = "qid_to_titles.csv"
with open(output_csv_path, mode="w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["QID", "Wikipedia_Title"])  # Header
    for qid, title in qid_title_pairs:
        writer.writerow([qid, title])

print(f"CSV of QIDs and titles written to {output_csv_path}")

Found 13971 unique QIDs in total.


Retrieving Wikipedia titles:   0%|          | 25/13971 [00:13<1:44:09,  2.23it/s]

No 'enwiki' sitelink for Q42712773


Retrieving Wikipedia titles:   3%|▎         | 432/13971 [03:29<1:30:09,  2.50it/s]

No 'enwiki' sitelink for Q7005377


...

No 'enwiki' sitelink for Q1364307


Retrieving Wikipedia titles:  99%|█████████▉| 13853/13971 [1:55:49<01:06,  1.77it/s]

No 'enwiki' sitelink for Q16145297


Retrieving Wikipedia titles: 100%|██████████| 13971/13971 [1:56:45<00:00,  1.99it/s]

Successfully retrieved 13891 titles.
CSV of QIDs and titles written to qid_to_titles.csv
